In [20]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.agents import create_agent

from langchain_groq import ChatGroq


  

import asyncio
import inspect
import os

In [27]:

# Build the MultiServerMCPClient configuration (do not start yet)
client = MultiServerMCPClient(
    {
        "math":{
            "command": "python",
            "args": ["mathserver.py"],
            "transport": "stdio"
        },
        # Weather tool removed for now to simplify debugging (start with math only)
        # "weather":{ 
        #     "transport": "streamable_http",
        #     "url": "http://localhost:8000/mcp"
        # }
    }
)

In [29]:

import os
from dotenv import load_dotenv
load_dotenv() 
# Put all awaited calls inside an async main() to avoid top-level await errors
async def main():
    # Safely set GROQ_API_KEY only when present (avoid setting None into os.environ)
    _groq_key = os.getenv("GROQ_API_KEY")
    if _groq_key:
        os.environ["GROQ_API_KEY"] = _groq_key
    else:
        print("Warning: GROQ_API_KEY not set; continuing without it")

    tools = await client.get_tools()
    # print loaded tools defensively in case tool objects differ
    try:
        tool_names = [getattr(tool, "name", repr(tool)) for tool in tools]
    except Exception:
        tool_names = [repr(tool) for tool in tools]

    model=ChatGroq(model="openai/gpt-oss-20b") #qwen-qwq-32b
    agent=create_agent(model, tools)   

    print("Invoking math tool...")
    try:
        math_call = agent.invoke({"messages":[{"role":"user","content": "what is (3+5) x 12?"}]})
        if inspect.isawaitable(math_call):
            math_response = await asyncio.wait_for(math_call, timeout=60)
        else:
            math_response = math_call
    except asyncio.TimeoutError:
        print("Math invoke timed out after 60s")
        return
    except Exception as e:
        print("Math invoke failed:", repr(e))
        return

    # Print a debug summary of the response
    print("math_response:", repr(math_response))
